In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Load environment variables
load_dotenv()

In [ ]:
# --- Configuration ---
VECTOR_STORE_PATH = "vector_store"

def format_docs(docs):
    """Helper function to format retrieved documents for the prompt context."""
    return "\n\n".join(
        f"Source: {doc.metadata.get('source_file', 'N/A')}\nContent: {doc.page_content}" 
        for doc in docs
    )

In [ ]:
def main():
    """Main function to run the comparison for a new feature."""
    if not os.path.exists(VECTOR_STORE_PATH):
        print("Error: Vector store not found. Please run 'ingest.py' first.")
        return

    # 1. Define the new feature to be analyzed
    feature_description = (
        "Feature Name: User Activity Tracker. "
        "This feature collects user click-stream data and location information to provide "
        "personalized recommendations. All collected data is considered Personally Identifiable Information (PII)."
    )
    print(f"--- Analyzing New Feature ---\n{feature_description}\n-----------------------------\n")

    # 2. Load the existing vector store and embedding model
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.load_local(VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
    retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # Retrieve top 5 relevant chunks

    # 3. Define the prompt template for the LLM
    prompt_template = """
    You are an expert legal compliance AI assistant. Your task is to analyze a new software feature 
    against a provided set of legal documents and identify potential compliance risks.

    **New Feature Description:**
    {feature_description}

    **Relevant Legal Information Retrieved from our Database:**
    {context}

    **Analysis Request:**
    Based on the feature description and the retrieved legal context ONLY, please provide a compliance analysis.
    1. Identify which laws are most relevant to this feature.
    2. Highlight specific parts of the feature that might have compliance implications.
    3. List any potential risks or areas that require further legal review.
    """
    
    prompt = ChatPromptTemplate.from_template(prompt_template)
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    
    # 4. Create the RAG chain
    chain = (
        {"context": retriever | format_docs, "feature_description": lambda x: x}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 5. Invoke the chain to get the analysis
    print("Generating compliance analysis...\n")
    analysis_result = chain.invoke(feature_description)

    print("--- Compliance Analysis Result ---")
    print(analysis_result)

In [ ]:
if __name__ == "__main__":
    main()